Notebook which will:
- Obtain alert photometry for named SN and create alert.
- Check whether this passes through a filter.

In [2]:
import requests, os
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.SimpleDecentFilter import SimpleDecentFilter
from ampel.contrib.hu.t0.RedshiftCatalogFilter import RedshiftCatalogFilter

from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier

import numpy as np

In [3]:
logger = AmpelLogger.get_logger()

In [4]:
# Access to the AMPEL data archive assumes an individual *archive token* which can be obtained from 
# https://ampel.zeuthen.desy.de/live/dashboard/tokens
# This currently builds on GitHub authorization for AmpelProject members.
token = os.environ["ARCHIVE_TOKEN"]

In [9]:
filename = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/ligo-healpix/broken_ampel_200115.csv"

# Name of ZTF objects to lookup
snnames = np.genfromtxt(filename, delimiter=",", dtype=None,encoding="UTF-8")

print(snnames)

['ZTF18acckpja' 'ZTF19aaevvnk' 'ZTF20aafqpdz' 'ZTF20aafmdyo'
 'ZTF20aafmdyo' 'ZTF20aafqtmj' 'ZTF20aafqpyx' 'ZTF20aafduuu'
 'ZTF20aafqtun' 'ZTF20aafqqqt' 'ZTF19aaevnir' 'ZTF20aafqpum'
 'ZTF20aafqvyc' 'ZTF20aafrftb' 'ZTF20aafqxxd' 'ZTF20aafrelr'
 'ZTF19aagymlm' 'ZTF20aaeodhh' 'ZTF20aafvbnj' 'ZTF19aamftdx'
 'ZTF20aafvasg' 'ZTF20aaetqwv' 'ZTF20aafvash' 'ZTF20aafukwm']


In [11]:
def api_to_alert(name, token, tags=['FilterTest']):
    """
    Retrieve the alert history of a SN and return all recreated alerts.    
    """
    
    # Setup connection
    endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/v3/object/{}/alerts?with_history=true".format(name)

    header = {"Authorization": "bearer "+token}

    response = requests.get(endpoint, headers=header )
    
    if not response.ok:
        print('... failed to get alert')
        return None
    
    return [ZiAlertSupplier.shape_alert_dict( a, tags ) for a in response.json()]

In [12]:
alert_array = []
for snname in snnames:
    alerts = api_to_alert(snname, token)
    print(f'{snname}: Found {len(alerts)} alerts.')
    alert_array.append(alerts)

ZTF18acckpja: Found 59
ZTF19aaevvnk: Found 8
ZTF20aafqpdz: Found 1
ZTF20aafmdyo: Found 26
ZTF20aafmdyo: Found 26
ZTF20aafqtmj: Found 4
ZTF20aafqpyx: Found 3
ZTF20aafduuu: Found 9
ZTF20aafqtun: Found 7
ZTF20aafqqqt: Found 8
ZTF19aaevnir: Found 4
ZTF20aafqpum: Found 3
ZTF20aafqvyc: Found 2
ZTF20aafrftb: Found 1
ZTF20aafqxxd: Found 3
ZTF20aafrelr: Found 1
ZTF19aagymlm: Found 4
ZTF20aaeodhh: Found 2
ZTF20aafvbnj: Found 1
ZTF19aamftdx: Found 28
ZTF20aafvasg: Found 1
ZTF20aaetqwv: Found 17
ZTF20aafvash: Found 1
ZTF20aafukwm: Found 12


Defining parameters for filter.

In [21]:
filter_config = {
    #"gaia_excessnoise_sig_max": 999,
    #"gaia_plx_signif": 3,
    #"gaia_pm_signif": 3,
    #"gaia_rs": 10,
    #"gaia_veto_gmag_max": 20,
    #"gaia_veto_gmag_min": 9,
    "min_ndet": 1, # should be 2
    "min_tspan": -1,
    "max_tspan": 365,
    #min_archive_tspan: -99
    "max_archive_tspan": 3,
    "min_drb": 0.3,
    "min_gal_lat": 0,
    "min_rb": 0.0,
    "min_sso_dist": 20,
    "ps1_confusion_rad": 3,
    "ps1_confusion_sg_tol": 0.1,
    "ps1_sgveto_rad": 1,
    "ps1_sgveto_th": 0.8,
    "max_fwhm": 5.5,
    # max_elong: 2
    "max_elong": 1.4,
    "max_magdiff": 1,
    "max_nbad": 2
}

In [30]:
t0filter = SimpleDecentFilter( **filter_config, logger=logger )
t0filter.post_init()


2023-05-25 11:22:53 SimpleDecentFilter:59 INFO
 Using min_ndet=1
 Using min_tspan=-1.0
 Using max_tspan=365.0
 Using min_archive_tspan=0.0
 Using max_archive_tspan=100000.0
 Using min_drb=0.3
 Using min_rb=0.0
 Using max_fwhm=5.5
 Using max_elong=1.4
 Using max_magdiff=1.0
 Using max_nbad=2
 Using min_sso_dist=20.0
 Using min_gal_lat=0.0
 Using ps1_sgveto_rad=1.0
 Using ps1_sgveto_th=0.8
 Using ps1_confusion_rad=3.0
 Using ps1_confusion_sg_tol=0.1


In [34]:

trigger_jd = 2458863.682951

In [43]:
def process(alerts):

    pass_array = []

    for alert in alerts:
        passes = True
        alert_jds = np.array(alert.get_values("jd", filters=t0filter.filter_pps))

        predetection_jds = [jd for jd in alert_jds if jd < trigger_jd]

        #if len(predetection_jds) > 0:
            #passes = False
            #pass_array.append(passes)
            #continue

        #pass_array.append(passes)
        #continue 

        foo = t0filter.process(alert)
        if not foo:
            #print(foo)
            passes = False
            pass_array.append(passes)
            continue

        pass_array.append(passes)
        #print(foo)

    print("total:", np.any(pass_array), pass_array)

In [44]:
for i, snname in enumerate(snnames):
    print(snname, end =" ")
    process(alert_array[i])

ZTF18acckpja total: True [True, True, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True]
ZTF19aaevvnk total: True [False, True, False, True, False, False, False, False]
ZTF20aafqpdz total: True [True]
ZTF20aafmdyo total: True [True, True, True, True, True, True, False, False, False, False, True, True, True, False, True, True, True, True, True, False, False, True, True, True, True, True]
ZTF20aafmdyo total: True [True, True, True, True, True, True, False, False, False, False, True, True, True, False, True, True, True, True, True, False, False, True, True, True, True, True]
ZTF20aafqtmj total: True [True, True, True, True]
ZTF20aafqpyx total: True [True, True, True]
ZTF20aafduuu total: True [True, T


2023-05-25 11:42:33 SimpleDecentFilter:212 INFO [elong=1.4753451347351074]

2023-05-25 11:42:33 SimpleDecentFilter:212 INFO [elong=1.5266457796096802]

2023-05-25 11:42:33 SimpleDecentFilter:212 INFO [elong=1.4788419008255005]

2023-05-25 11:42:33 SimpleDecentFilter:191 INFO [isdiffpos=f]

2023-05-25 11:42:33 SimpleDecentFilter:191 INFO [isdiffpos=f]

2023-05-25 11:42:33 SimpleDecentFilter:191 INFO [isdiffpos=f]

2023-05-25 11:42:33 SimpleDecentFilter:191 INFO [isdiffpos=f]

2023-05-25 11:42:33 SimpleDecentFilter:191 INFO [isdiffpos=f]

2023-05-25 11:42:33 SimpleDecentFilter:202 INFO [drb=0.1122591495513916]

2023-05-25 11:42:33 SimpleDecentFilter:207 INFO [fwhm=5.789999961853027]

2023-05-25 11:42:33 SimpleDecentFilter:207 INFO [fwhm=6.639999866485596]

2023-05-25 11:42:33 SimpleDecentFilter:207 INFO [fwhm=5.75]

2023-05-25 11:42:33 SimpleDecentFilter:207 INFO [fwhm=5.670000076293945]

2023-05-25 11:42:33 SimpleDecentFilter:207 INFO [fwhm=5.739999771118164]

2023-05-25 11:42:33 Simpl

total: True [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
ZTF20aafvasg total: True [True]
ZTF20aaetqwv total: True [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
ZTF20aafvash total: True [True]
ZTF20aafukwm total: True [True, True, False, False, True, True, True, True, True, True, True, True]



2023-05-25 11:42:34 SimpleDecentFilter:207 INFO [fwhm=5.75]

2023-05-25 11:42:34 SimpleDecentFilter:207 INFO [fwhm=5.570000171661377]


In [25]:
for a in alerts:
    latest = a.datapoints[0]
    # drb
    try:
        print('drb', latest['drb'])
    except KeyError:
        print('no drb')
    # rb
    try:
        print('rb', latest['rb'])
    except KeyError:
        print('no rb')
    # isdiffpos
    try:
        print('isdiffpos', latest['isdiffpos'])
    except KeyError:
        print('no isdiffpos')


drb 0.9851456880569458
rb 0.618571400642395
isdiffpos t
drb 0.9999991655349731
rb 0.8242856860160828
isdiffpos t
drb 0.9996000528335571
rb 0.46714285016059875
isdiffpos t
drb 0.9997393488883972
rb 0.5571428537368774
isdiffpos t
drb 0.9999111890792847
rb 0.8428571224212646
isdiffpos t
drb 0.9999647736549377
rb 0.8357142806053162
isdiffpos t
drb 0.9970144033432007
rb 0.7614285945892334
isdiffpos t
drb 0.9983726143836975
rb 0.6842857003211975
isdiffpos t
drb 0.9981058835983276
rb 0.7914285659790039
isdiffpos t
drb 0.9999994039535522
rb 0.9314285516738892
isdiffpos t
drb 0.9996236562728882
rb 0.9128571152687073
isdiffpos t
drb 0.9999997615814209
rb 0.9385714530944824
isdiffpos t


In [ ]:
alerts[0].datapoints

### Explore the RedshiftFilter

In [ ]:
zfilter_config = {
    "catalog_match_radius": 30,
    "min_z": 0.002,
    "max_z": 0.025,
    "max_tspan": 5.,        # Max total detection age in alert
    "max_archive_tspan": 5.,        # Max total detection age in IPAC DB
    "min_rb": 0.3,             # real bogus score
    "min_drb": 0.995,         # deep learning real bogus score 
    # Should be set during proper install
    "resource": {"ampel-ztf/catalogmatch":"https://ampel.zeuthen.desy.de/api/catalogmatch/"},
}

In [ ]:
zfilter = RedshiftCatalogFilter( **zfilter_config, logger=logger )
zfilter.post_init()

In [ ]:
for alert in alerts:
    foo = zfilter.process(alert)
    print(alert.datapoints[0]['jd'], foo)

In [ ]:
alerts[0].datapoints[0]['jd']